# Wrangle OpenStreetMap Data Project
***

## Map Area

I chose Salt Lake City, UT. I grew up here and know the area pretty well, but was interested in seeing what I would find in the OpenStreetMap data.

https://www.openstreetmap.org/relation/198770

## Problems Encountered in the Map

While exploring the data I found the below problems:

* Street names were abbreviated inconsistently, for example Avenue was written 3 different ways on different streets (Ave, Ave., Avenue).


* Postcodes were not consistent, for example some postcodes included the state abbreviation before the code (UT 84116).


* State names were also inconsistent with their abbreviation, Utah was written 4 different ways (Utah, UT, Ut, ut).


* There was an entry for an address in Cincinnati, Ohio. It was the only entry with a city, state, or zipcode outside of the Salt Lake City area. It somehow was errouneoulsy added to the dataset.

## Data Cleansing

I wrote the below 'update_name' function to programmaticaly fix the first data issues describe above. 

```
def update_name(name, tag_key, tag_type):
    if tag_key == 'street' and tag_type == 'addr':
        name = name.split(' ')
        if name[len(name)-1] in street_mapping.keys():
            name[len(name)-1] = street_mapping[name[len(name)-1]]
        name =  ' '.join(name)
        return name
    elif tag_key == 'state' and tag_type == 'addr':
        if name in state_mapping.keys():
            name = state_mapping[name]
        return name
    elif tag_key == 'postcode' and tag_type == 'addr':
        m = postcode_check.search(name)
        name = m.group()
        return name
    else:
        return name
```

The street names are cleansed using this mapping dictionary:
```
street_mapping = { "Ave": "Avenue",
            "Ave.": "Avenue",
            "Blvd": "Boulevard",
            "Dr": "Drive",
            "E": "East",
            "N": "North",
            "Pl": "Place",
            "S": "South",
            "St": "Street",
            "W": "West",
            "W.": "West"}
```

The states are cleansed using this mapping dictionary:
```
state_mapping = { "Utah": "UT",
                "Ut": "UT",
                "ut": "UT"}
```

The postcodes are cleansed using regex to just return items that are either 5 digit integers or 5 digit integers followed by a '-' and another 4 digit integer:
```
postcode_check = re.compile('\d{5}(-\d{4})?')
```

## Overview of the Data

Below are queries that describe the dataset that has been cleansed and loaded into the database.

##### Size of the file
The SLC.osm file is 85 MB. 

##### Number of unique users
```
SELECT COUNT(DISTINCT user) as total_users 
FROM (
    SELECT user FROM nodes
    UNION ALL
    SELECT user FROM ways
) a;

There are 566 unique users
```

##### Number of Node and Way edits
```
SELECT COUNT(user) as total_edits
FROM (
    SELECT user FROM nodes
    UNION ALL
    SELECT user FROM ways
) a;

There were 461,566 total edits.
```

##### Number of Node, Way, and Total edits by each user, with the percent of edits each user made of the total edits (the dataframe below shows the top 5)
```
SELECT nodes.user
    ,node_edits
    ,way_edits
    ,node_edits + way_edits AS total_edits
    ,(node_edits + way_edits) * 1.0 / 
        (SELECT COUNT(user) FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways)) * 100 
     AS percent_of_total
FROM (SELECT user, count(*) node_edits FROM nodes GROUP BY user) nodes
JOIN (SELECT user, count(*) way_edits FROM ways GROUP BY user) ways
ON nodes.user = ways.user
ORDER BY total_edits DESC;

          node_edits  way_edits  total_edits  percent_of_total
user                                                          
chadbunn      160129      18515       178644             38.70
butlerm        44420       2416        46836             10.15
mvexel         35084       4033        39117              8.47
wrk3           31582       5153        36735              7.96
GaryOSM        18292       3614        21906              4.75
```

##### Percent of total edits made by the top 5 users - 70% of edits came from top 5, 38% from the top user

```
edits_df.sort_values('percent_of_total',ascending=False)['percent_of_total'].head().plot(kind='bar')
```

<img src="graph.png">

##### Top 10 amenities
```
SELECT value as amenity, COUNT(*) as [count]
FROM nodes_tags
WHERE key = 'amenity'
GROUP BY value
ORDER BY [count] DESC
LIMIT 10;

                  count
amenity                
restaurant          417
place_of_worship    190
fast_food           147
parking             134
bench                99
cafe                 75
school               67
fuel                 53
bank                 52
toilets              42
```

##### Restaurants with more than two locations
```
SELECT nodes_tags.value as restaurant, COUNT(*) as [count]
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='name'
GROUP BY nodes_tags.value
HAVING [count] > 2
ORDER BY [count] DESC;

            count
restaurant       
Costa Vida      5
Pizza Hut       4
Cafe Rio        3
```

## Additional Ideas

* One area for imporovement could be fixing the way the 'update_name' function works in regards to street name updating. It only looks at the last word of the street name and updates those, but it doesn't fix any abbreviations of street names that don't fall in the last word of the string. For example if a street is listed as '300 W Temple' it wouldn't update the W to West even though that is in the mapping dictionary. This wouldn't be too difficult to add, it would just require looping through all words in the string. However, it would require making sure that a W is the letter of a Suite or an apartment or something similar. 


* Looking at the list of restaurants with more than 2 locations, I am thinking there are quite a few things missing. You would expect to see many chain restaurants like Chilli's, IHOP, etc. The list of restaurants is probably far from complete and could use some work to finish out. This might be somewhat manual, but it could be good if it could be automated somehow.